In [19]:
import pandas as pd
import numpy as np
from dynaconf import LazySettings
from dynaconf.utils.boxing import DynaBox
from typing import List
import os

In [20]:
config_file = "C://workspaces//learning//fantasAi_football//config//conf.yaml"
config_mode = 'default'

In [21]:
def data_ingestion_basics(params: DynaBox, table: str, duplicate_key_err:str='raise') -> pd.DataFrame:
    import os 
    
    """Performs some basic harmonization steps on a input dataframe.

    Note that after the basic harmonization the columns will have the names
    stated in settings.COLS, and not those appearing in the input data.

    Args:
        table (pd.DataFrame): id of the table to lead, it must appear in
            config file
        duplicate_key_err ('raise' or 'drop'): how to handle duplicated key
        errors. If 'raise' an error will be return in case of duplicated keys,
        if 'drop' all duplicated rows will be dropped from data.

    Raises:
        ValueError: If table registry dataframe keys are missing
        KeyError: If table registry dataframe keys are duplicated

    Returns:
        pd.DataFrame: dataframe in input, harmonized
    """
    # Load parameters
    t_par = params[table]
    t_cols_dict = t_par["COLS"].to_dict()
    keys: List[str] = [t_par["COLS"][c] for c in t_par.KEY]
    cols: List[str] = list(t_cols_dict.values())
    dtype_map = {
        c_name: params["FEATURES"]["DTYPES"][c]
        for c, c_name in t_cols_dict.items()
    }
    name_conversion = {
        c_from: params["FEATURES"][c] for c, c_from in t_cols_dict.items()
    }
    
    path = os.path.join(
        params["PATHS"]["ROOT_FOLDER"], 
        params["PATHS"]["INPUT"]["FOLDER"],
         params["PATHS"]["INPUT"][table]
    )
    data: pd.DataFrame = pd.read_csv(path)

    # Keep only selected columns
    data = data[cols]

    # Ugly but functional call to ensure correct type conversion
    data = data.convert_dtypes().astype(dtype_map, errors='ignore').convert_dtypes()  # type: ignore

    # Ensure that the product registry dataframe keys are valid
    if not data[keys].notna().all(axis=1).all():
        raise ValueError(f"{table} dataframe keys are missing")
    if not data.value_counts(keys).eq(1).all():
        if duplicate_key_err == 'raise':
            raise KeyError(f"{table} keys are duplicated")
        else:
            print(f"WARN: dupliated keys will be removed from {table}")
            data = data.drop_duplicates(subset=keys, keep=False)

    # Sort product product registry by DIVISION,PRODUCT and reset index
    data = data.sort_values(keys).reset_index(drop=True)

    data = data.rename(columns=name_conversion)

    if "FILTER" in params:
        if table in params["FILTER"]:
            for column, values in params["FILTER"].get(table).to_dict().items():
                column_name = params["FEATURES"].get(column)
                data = data.loc[data[column_name].isin(values)]

    return data

def clean_multiindex_agg(frame: pd.DataFrame) -> pd.DataFrame:
    frame.columns = list(map('||'.join, frame.columns.values))
    frame = frame.rename(
        columns={
            "date||max": "_date_max",
            "date||min": "_date_min"}
            )
    frame.columns = [c.replace("||sum", "") for c in frame.columns]
    frame.columns = [c.replace("||count", "") for c in frame.columns]
    frame.columns = [c.replace("||", "") for c in frame.columns]

    return frame

In [22]:
params = LazySettings(settings_files=[config_file])
params = params[config_mode]

In [23]:
players = data_ingestion_basics(params, "PLAYERS")
clubs = data_ingestion_basics(params, "CLUBS")
competitions = data_ingestion_basics(params, "COMPETITIONS")
games = data_ingestion_basics(params, "GAMES")
games[params["FEATURES"]["DATE"]] = pd.to_datetime(games[params["FEATURES"]["DATE"]])
#international_competitions_stats = data_ingestion_basics(params, "INTERNATIONAL_COMPETITIONS_STATS")
player_valuations = data_ingestion_basics(params, "PLAYER_VALUATIONS")
players = data_ingestion_basics(params, "PLAYERS")
appearances = data_ingestion_basics(params, "APPEARANCES")
votes_ita = data_ingestion_basics(params, "VOTES_ITA", duplicate_key_err='drop')

WARN: dupliated keys will be removed from VOTES_ITA


## De-normalize data


Find a player appearences for a given year.


In [24]:
# For some reason competition is specified also on appearances table, this
# is not necessary and annoying since it generates suffixed names when merged.
apps_sel = appearances.drop(columns=[params["FEATURES"]["COMPETITION"]])

# Get a dataframe with all details about the game in which the player appears
app_game = apps_sel.merge(games, on=[params["FEATURES"]["GAME"]])

# Create seasonal players appearances data
app_game = app_game.groupby(
    [
        params["FEATURES"]["PLAYER"],
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["COMPETITION"],
        params["FEATURES"]["CLUB"],
    ],
    as_index=False,
).agg(
    {
        params["FEATURES"]["GOALS"]: "sum",
        params["FEATURES"]["ASSISTS"]: "sum",
        params["FEATURES"]["MINUTES_PLAYED"]: "sum",
        params["FEATURES"]["DATE"]: ["max", "min"],
    }
)

app_game = clean_multiindex_agg(app_game)

# How many minutes the player played
app_game = app_game.rename(
    columns={params["FEATURES"]["GAME"]: "_minutes_played"}
)
# How many weeks the player stayed in the club as difference from the
# first appearance to the last (we can not now when the player actually
# joined the club)
app_game["_permanence_week"] = (
    (app_game["_date_max"] - app_game["_date_min"]).dt.days / 7
) + 1

app_game = app_game.drop(columns=["_date_max", "_date_min"])

# Filter out rows for played that stayed less than a certain number of weeks
prev = len(app_game)
app_game = app_game.loc[
    app_game["_permanence_week"] > params["SETTINGS"]["MIN_PERMANENCE_WEEKS"]
]
print(
    f"Filtered out {prev - len(app_game)}/{prev} player seasons with "
    + " too short permancence time."
)


Filtered out 50852/112222 player seasons with  too short permancence time.


In [25]:
# Add information about total minutes available in the competition
total_minutes = games.groupby(
    [
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["COMPETITION"],
        params["FEATURES"]["HOME_CLUB"],
    ],
    as_index=False,
).agg({
        params["FEATURES"]["GAME"]: "count",        
        params["FEATURES"]["DATE"]: ["max", "min"],
    })

total_minutes = clean_multiindex_agg(total_minutes)

total_minutes["_season_weeks"] = (total_minutes["_date_max"] - total_minutes["_date_min"]).dt.days/7

total_minutes = total_minutes.groupby(
    [params["FEATURES"]["SEASON"], params["FEATURES"]["COMPETITION"]],
    as_index=False,
).max()
total_minutes["_minutes_available"] = total_minutes[params["FEATURES"]["GAME"]] * params["SETTINGS"]["MINUTES_PER_GAME"] * 2

total_minutes = total_minutes.drop(columns=["_date_max", "_date_min"])

In [26]:
# Attach club, minutes played and permanence in weeks to player dataframe
players_team = players.merge(app_game, on=[params["FEATURES"]["PLAYER"]])

# Attach total minutes available in a competition
players_onfield = players_team.merge(
    total_minutes,
    on=[params["FEATURES"]["COMPETITION"], params["FEATURES"]["SEASON"]],
)

# Calculate the "ON_FIELD_INDEX" feature, and clean up all the temp fields

# How much of the season did the player spent in the club
players_onfield["_season_played"] = (
    players_onfield["_permanence_week"] / players_onfield["_season_weeks"]
)
players_onfield["_season_played"] = players_onfield["_season_played"].clip(
    upper=1
)

# If a players played only for one club, in one competition and one season
# è played the entire season there
players_onfield["_season_teams"] = players_onfield.groupby(
    [
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["COMPETITION"],
        params["FEATURES"]["PLAYER"],
    ]
).transform("count")[params["FEATURES"]["GAME"]]
players_onfield.loc[players_onfield["_season_teams"] <= 1, "_season_played"] = 1


players_onfield[params["FEATURES"]["ON_FIELD_INDEX"]] = (
    players_onfield["minutes_played"] / players_onfield["_minutes_available"]
).div(players_onfield["_season_played"])

# When player changed team one or more times in the same season, the season
# played measure may be imprecise, leading to a ON_FIELD_INDEX greater than
# one
players_onfield[params["FEATURES"]["ON_FIELD_INDEX"]] = players_onfield[
    params["FEATURES"]["ON_FIELD_INDEX"]
].clip(upper=1)

players_onfield[params["FEATURES"]["ASSISTS_RATIO"]] = (
    players_onfield[params["FEATURES"]["ASSISTS"]].div(
        players_onfield["minutes_played"]
    )
    * params["SETTINGS"]["MINUTES_PER_GAME"]
)
players_onfield[params["FEATURES"]["GOALS_RATIO"]] = (
    players_onfield[params["FEATURES"]["GOALS"]].div(
        players_onfield["minutes_played"]
    )
    * params["SETTINGS"]["MINUTES_PER_GAME"]
)
players_onfield[params["FEATURES"]["FANTA_POINTS_RATIO"]] = (
    players_onfield[params["FEATURES"]["ASSISTS"]]
    * params["SETTINGS"]["ASSIST_FANTA_POINTS"]
    + players_onfield[params["FEATURES"]["GOALS"]]
    * params["SETTINGS"]["GOAL_FANTA_POINTS"]
).div(players_onfield["minutes_played"]) * params["SETTINGS"][
    "MINUTES_PER_GAME"
]

for ratio_cols in [
    params["FEATURES"]["ON_FIELD_INDEX"],
    params["FEATURES"]["ASSISTS_RATIO"],
    params["FEATURES"]["GOALS_RATIO"],
    params["FEATURES"]["FANTA_POINTS_RATIO"],
]:
    players_onfield[ratio_cols] = players_onfield[ratio_cols].replace(np.inf, 0)


players_onfield = players_onfield.drop(
    columns=[
        "game_id",
        # "_season_weeks",
        # "_season_played",
        # "_season_teams",
        # "_permanence_week",
        # "_minutes_available",
        "home_club_id",
    ]
)


Load player value for each season


In [27]:
# Define a start date for a season (player value will be read from this)
players_onfield["year"] = players_onfield[params["FEATURES"]["SEASON"]]
players_onfield["month"] = 9
players_onfield["day"] = 1
players_onfield["_season_starts"] = pd.to_datetime(players_onfield[["year", "month", "day"]])
players_onfield = players_onfield.drop(columns=["year", "month", "day"])

In [28]:
# Load player valuation closest to the start of the season
players_vals = players_onfield.merge(player_valuations, on=[params["FEATURES"]["PLAYER"]])
players_vals["_date_diff"] = (players_vals[params["FEATURES"]["DATE"]] - players_vals["_season_starts"]).dt.days
players_vals = players_vals.loc[players_vals["_date_diff"] < 0]

pv_key = [params["FEATURES"]["PLAYER"], params["FEATURES"]["SEASON"]]
players_vals = players_vals[players_vals.groupby(pv_key)["_date_diff"].transform(max) == players_vals["_date_diff"]]
players_lost = len(players_team) - len(players_vals)
players_vals = players_vals.drop(columns=["_date_diff", "_season_starts", params["FEATURES"]["DATE"]])
print(f"{players_lost}/{len(players_team)} players/seasons has been lost when trying to calculate the value.")

1473/61370 players/seasons has been lost when trying to calculate the value.


In [29]:
players_vals

,player_id,name,pretty_name,country_of_citizenship,date_of_birth,position,sub_position,foot,height_in_cm,season,...,_permanence_week,_season_weeks,_minutes_available,_season_played,_season_teams,on_field_index,assist_ratio,goals_ratio,fanta_points_ratio,market_value
19,10,miroslav-klose,Miroslav Klose,Germany,09-06-78,Attack,attack - Centre-Forward,Right,184,2014,...,39.428571,38.428571,720,1.0,1,0.630556,0.396476,0.594714,2.180617,900000
42,10,miroslav-klose,Miroslav Klose,Germany,09-06-78,Attack,attack - Centre-Forward,Right,184,2014,...,40.000000,39.142857,3420,1.0,1,0.53655,0.343324,0.637602,2.256131,900000
67,10,miroslav-klose,Miroslav Klose,Germany,09-06-78,Attack,attack - Centre-Forward,Right,184,2015,...,17.000000,32.857143,1440,1.0,1,0.114583,0.0,0.545455,1.636364,900000
90,10,miroslav-klose,Miroslav Klose,Germany,09-06-78,Attack,attack - Centre-Forward,Right,184,2015,...,31.000000,38.142857,3420,1.0,1,0.397661,0.463235,0.463235,1.852941,900000
100,101213,pedro-obiang,Pedro Obiang,Equatorial Guinea,27-03-92,Midfield,midfield - Central Midfield,Right,186,2014,...,15.571429,38.428571,720,1.0,1,0.215278,0.0,0.0,0.0,7200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1362972,73427,caleb-patterson-sewell,Caleb Patterson Sewell,United States,20-05-87,Goalkeeper,Goalkeeper,Right,190,2014,...,13.000000,19.285714,540,1.0,1,0.666667,0.0,0.0,0.0,360000
1362995,256669,nicolae-milinceanu,Nicolae Milinceanu,Moldova,01-08-92,Attack,attack - Centre-Forward,Both,189,2020,...,14.857143,29.571429,720,1.0,1,0.433333,0.0,0.288462,0.865385,270000
1363000,297013,pantelis-panourgias,Pantelis Panourgias,Greece,13-04-98,Defender,Defender - Centre-Back,Left,190,2020,...,11.857143,29.571429,720,1.0,1,0.083333,0.0,0.0,0.0,23000
1363011,353609,titas-krapikas,Titas Krapikas,Lithuania,03-01-99,Goalkeeper,Goalkeeper,Right,194,2020,...,14.142857,17.857143,720,1.0,1,0.583333,0.0,0.0,0.0,315000


In [30]:
players_vals = players_vals.sort_values(by=pv_key)
players_vals[params["FEATURES"]["VALUE_DELTA"]] = players_vals[params["FEATURES"]["MARKET_VALUE"]].diff()
players_vals[params["FEATURES"]["VALUE_DELTA"]] = players_vals[params["FEATURES"]["VALUE_DELTA"]]/players_vals[params["FEATURES"]["MARKET_VALUE"]]

# Calculate club value
players_vals["_club_value"] = players_vals.groupby(
    [params["FEATURES"]["CLUB"], params["FEATURES"]["SEASON"]]
)[params["FEATURES"]["MARKET_VALUE"]].transform("sum")

# Express club value as ration from media club value of same competition
players_vals["_median_club_value"] = players_vals.groupby(
    [params["FEATURES"]["COMPETITION"], params["FEATURES"]["SEASON"]]
)["_club_value"].transform("median")
players_vals[params["FEATURES"]["CLUB_VALUE_RATIO"]] = (
    players_vals["_club_value"] / players_vals["_median_club_value"]
)

# Express players value as ratio from median player value of same competition
players_vals["_player_median_value"] = players_vals.groupby(
    [params["FEATURES"]["COMPETITION"], params["FEATURES"]["SEASON"]]
)[params["FEATURES"]["MARKET_VALUE"]].transform("sum")
players_vals[params["FEATURES"]["MARKET_VALUE_RATIO"]] = (
    players_vals[params["FEATURES"]["MARKET_VALUE"]]
    / players_vals["_player_median_value"]
)

# Express players narrow replaceability as ratio from median player value of 
# same team and sub-position
players_vals["_team_sub_position_value"] = players_vals.groupby(
    [
        params["FEATURES"]["CLUB"],
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["SUB_POSITION"],
    ]
)[params["FEATURES"]["MARKET_VALUE"]].transform("sum")
players_vals[params["FEATURES"]["MARKET_REPL_NARROW"]] = (
    players_vals[params["FEATURES"]["MARKET_VALUE"]]
    / players_vals["_team_sub_position_value"]
)

# Express players wide replaceability as ratio from median player value of 
# same team and position
players_vals["_team_position_value"] = players_vals.groupby(
    [
        params["FEATURES"]["CLUB"],
        params["FEATURES"]["SEASON"],
        params["FEATURES"]["POSITION"],
    ]
)[params["FEATURES"]["MARKET_VALUE"]].transform("median")
players_vals[params["FEATURES"]["MARKET_REPL_WIDE"]] = (
    players_vals[params["FEATURES"]["MARKET_VALUE"]]
    / players_vals["_team_position_value"]
)

players_vals = players_vals.drop(
    columns=[
    '_club_value', 
    '_median_club_value',
    "_player_median_value",
    "_median_club_value",
    "_player_median_value",
    "_team_position_value",
    "_team_sub_position_value",
    ]
    )

Add Club name

In [31]:
clubs_sel = clubs[[params["FEATURES"]["CLUB"], params["FEATURES"]["CLUB_PRETTY_NAME"]]]
tm_dataset = players_vals.merge(clubs_sel, on=[params["FEATURES"]["CLUB"]])

Get dummy variables for preferred foot and side

In [32]:
# Get dummy foot columns
foot_feature = params["FEATURES"]["FOOT"]
for lit_pos, tuple_pos in params["SETTINGS"]["POSITION_INDEXING"].items():
    position = tm_dataset[params["FEATURES"]["SUB_POSITION"]]
    tm_dataset.loc[position == lit_pos, params["FEATURES"]["POSITION_INDEX"]] = tuple_pos[0]
    tm_dataset.loc[position == lit_pos, params["FEATURES"]["SIDE_INDEX"]] = tuple_pos[1]

one_hot = pd.get_dummies(tm_dataset[params["FEATURES"]["FOOT"]], prefix=foot_feature)
tm_dataset_ix = tm_dataset.join(one_hot)
# NA is like both
tm_dataset_ix[f"{foot_feature}_Both"] = tm_dataset_ix[f"{foot_feature}_Both"] + tm_dataset_ix[f"{foot_feature}_<NA>"]
tm_dataset_ix = tm_dataset_ix.drop(columns=[f"{foot_feature}_<NA>"])

# Get dummy side columns
one_hot = pd.get_dummies(tm_dataset_ix[params["FEATURES"]["SIDE_INDEX"]].astype(int), prefix="side")
tm_dataset_ix = tm_dataset_ix.join(one_hot)

In [ ]:
# Do some harmonization and save votes ita
votes_ita[params["FEATURES"]["SEASON"]] = votes_ita[params["FEATURES"]["SEASON"]] + 2000
#votes_ita[params["FEATURES"]["TEAM"]].str.replace(params["VOTES_ITA"]["TEAM_TRANSLATOR"])
path = os.path.join(
    params["PATHS"]["ROOT_FOLDER"],
    f"{params['PATHS']['STAGES']['VOTES_ITA']}.pkl"
    )
votes_ita.to_pickle(path)

In [ ]:
tm_dataset_ix.iloc[0]

player_id                                      10
name                               miroslav-klose
pretty_name                        Miroslav Klose
country_of_citizenship                    Germany
date_of_birth                            09-06-78
position                                   Attack
sub_position              attack - Centre-Forward
foot                                        Right
height_in_cm                                  184
season                                       2014
competition_id                                CIT
club_id                                       398
goals                                           3
assists                                         2
minutes_played                                454
_permanence_week                        39.428571
_season_weeks                           38.428571
_minutes_available                            720
_season_played                                1.0
_season_teams                                   1


In [ ]:
path = os.path.join(
    params["PATHS"]["ROOT_FOLDER"],
    f"{params['PATHS']['STAGES']['TM_DATASET']}.pkl"
    )
tm_dataset_ix.to_pickle(path)